In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import os
import pickle
import math
#from utils import kits
import time
import random 
import numpy as np
import tensorflow as tf
import scipy.io
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops
#from skopt.utils import use_named_args
#from skopt import gp_minimize
#from skopt.space import Real, Categorical, Integer

from sklearn import preprocessing


In [ ]:
def eulerPlant(Cx,Ca,Cp,F,Tsamp):
    #next states for plant
    #Define constants
    mumax =40
    Ks =2e-1
    Yx =0.04
    Yp =0.8
    pstar=1.8
    Ca0=10.0

    kd= 1e-2;
    aa = 5.5;
    bb = 8e-1;
    
        
    if Cp>pstar:
       mu = 0
    else:
       mu = mumax*(Ca/(Ca+Ks))*(1-Cp/pstar)**3

        

    #Rate Definitions
    fcx=(mu-kd)*Cx - F*Cx
    fcp=aa*fcx +bb*Cx- F*Cp
    fca=-(1/Yx)*fcx - (1/Yp)*fcp + F*(Ca0-Ca)
   
    Cx_kplusone = Cx + Tsamp*fcx
    Ca_kplusone = Ca + Tsamp*fca
    Cp_kplusone = Cp + Tsamp*fcp
    
    
    return Cx_kplusone, Ca_kplusone,Cp_kplusone

In [ ]:
def eulerModel(Cx,Ca,Cp,F,Tsamp):
    #next states from model (original)
    #Define constants
    mumax =25.0#
    Ks =0.8
    Yx =0.05 #
    Yp =0.8 # 
    Ca0=10.0

    kd= 1e-13; #
    aa = 3.8;
    bb = 0.05;
    pstar=0.5

    np = 0.95;
    
    if Cp>pstar:
       mu = 0
    else:
       mu = mumax*(Ca/(Ca+Ks))*(1-Cp/pstar)


    #Rate Definitions
    fcx=(mu-kd)*Cx - F*Cx
    fcp=aa*fcx  - F*Cp
    fca=-(1/Yx)*fcx - (1/Yp)*fcp + F*(Ca0-Ca)
   
    Cx_kplusone = Cx + Tsamp*fcx
    Ca_kplusone = Ca + Tsamp*fca
    Cp_kplusone = Cp + Tsamp*fcp
    
    
    return Cx_kplusone, Ca_kplusone,Cp_kplusone

In [ ]:
def ModelSimulator(Tsamp,Nsamp,F):
    """
    Returns a trajectory of the HF model given some flow rate sequence
    """
    tsize = len(F)+1
    Cx_model = np.zeros((tsize,1))
    Ca_model = np.zeros((tsize,1))
    Cp_model = np.zeros((tsize,1))
    Cx_model[0]=0.2;Ca_model[0]=15;Cp_model[0]=0

    for k in range(len(F)):
          Cx_model[k+1], Ca_model[k+1], Cp_model[k+1] = eulerModel(Cx_model[k],Ca_model[k],Cp_model[k],F[k],Tsamp)
    return Cx_model, Ca_model, Cp_model

In [ ]:
def ResnetModelSimulator(Tsamp,Nsamp,F,DropModel):
    """
    Returns a trajectory of the HF model given some flow rate sequence
    """
    tsize = len(F)+1
    Cx_resnet = np.zeros((tsize,1))
    Ca_resnet = np.zeros((tsize,1))
    Cp_resnet = np.zeros((tsize,1))
    Cx_resnet[0]=0.2;Ca_resnet[0]=15;Cp_resnet[0]=0
    # xpr = np.array([x0,alpha[0],alpha[1],alpha[2],alpha[3],delta])
    #print(xpr)
    #xprn = xpr.reshape(1,6)
    #xnew = DropModel.predict(xprn)

    for k in range(len(F)):
        xpr = np.array([F[k],Cx_resnet[k],Ca_resnet[k],Cp_resnet[k]])
        xprn = xpr.reshape(1,4)
        #    xnew = DropModel.predict(xprn)
        futurep = DropModel.predict(xprn)
        Cx_resnet[k+1] = futurep[0][0]
        Ca_resnet[k+1] = futurep[0][1]
        Cp_resnet[k+1] = futurep[0][2]

    return Cx_resnet, Ca_resnet, Cp_resnet

In [ ]:
def PlantSimulator(Tsamp,Nsamp,F):
    """
    Returns a trajectory of the HF model given some flow rate sequence
    """
    tsize = len(F)+1
    Cx_plant = np.zeros((tsize,1))
    Ca_plant = np.zeros((tsize,1))
    Cp_plant = np.zeros((tsize,1))
    Cx_plant[0]=0.2;Ca_plant[0]=15;Cp_plant[0]=0

    for k in range(len(F)):
          Cx_plant[k+1], Ca_plant[k+1], Cp_plant[k+1] = eulerPlant(Cx_plant[k],Ca_plant[k],Cp_plant[k],F[k],Tsamp)
    return Cx_plant, Ca_plant, Cp_plant

In [ ]:
random.seed()
F=[]
Nsamp = 30
F1 = np.random.uniform(low=5, high=15, size=(Nsamp,))

In [ ]:
def HF_Model_Data(num,Nsamp): 

    random.seed(num)
    F=[]
    tarr=[]
    Tsamp = 0.005
    F = np.random.uniform(low=5, high=15, size=(Nsamp,))
    tarr = np.zeros((Nsamp,1))
    Cx_model,Ca_model,Cp_model = ModelSimulator(Tsamp,Nsamp,F)

    for j in range(Nsamp):
        tarr[j] = j*Tsamp

    tarr = np.append(tarr, np.repeat(np.nan, 1))
    F = np.append(F, np.repeat(np.nan, 1))
    return F,Cx_model,Ca_model,Cp_model

In [ ]:
def iterative_model(num,Nsamp):
    
    Xdata =np.zeros((Nsamp,4))
    Ydata =np.zeros((Nsamp,3))
    F,Cx_model,Ca_model,Cp_model = HF_Model_Data(num,Nsamp)

    for i in range(len(F)-1):
        Xdata[i,:] = [F[i],Cx_model[i],Ca_model[i],Cp_model[i]]
        Ydata[i,:] = [Cx_model[i+1],Ca_model[i+1],Cp_model[i+1]]
    
    return Xdata, Ydata



In [ ]:
Nruns=80;
Nsamp=30; #Time steps

X =np.zeros((Nruns,Nsamp,4))
Y =np.zeros((Nruns,Nsamp,3))

num = random.randint(1, 50000000)
random.seed(num)

for j in range(Nruns):
    X[j,:,:],Y[j,:,:]= iterative_model(num,Nsamp)
X.shape

(80, 30, 4)

In [ ]:
Xdata = X.reshape(Nruns*Nsamp,4)  #[Flow Rate, Cx_past, Ca_past, Cp_past]


In [ ]:
Xdnn = Xdata[0:70*Nsamp,:]
Xdnn.shape

(2100, 4)

In [ ]:
Xval = Xdata[50*Nsamp:,:]
Xval.shape

(900, 4)

In [ ]:
Ydata = Y.reshape(Nruns*Nsamp,3)  #[Ca_future, Cb_future, Tr_future]

In [ ]:
Ydnn = Ydata[0:70*Nsamp,:]
Ydnn.shape

(2100, 3)

In [ ]:
Yval = Ydata[50*Nsamp:,:]
Yval.shape

(900, 3)

In [ ]:
def swish(x, beta=1):
    return x * sigmoid(beta * x)

In [ ]:
def get_model_layers(nodes, layers, activation, learning_rate, training=None):
    if activation == 'swish':
        activation = swish
    else:
        activation = activation

    inputs = tf.keras.layers.Input(shape=(in_dim,))
  #  xadd = (np.array(inputs[:, :out_dim])).reshape(None,1)
   # print(inputs[:, :out_dim].shape)
    #print(xadd.shape)
    x = tf.keras.layers.Dense(units=nodes, activation=activation)(inputs)
    for i in range(layers - 1):  # at least one layer which has the parameter input as input
        x = tf.keras.layers.Dense(units=nodes, activation=activation)(x)

    #x = tf.keras.layers.Dropout(rate=0.1)(x, training=training)
    outputs = tf.keras.layers.Dense(out_dim, activation="linear")(x)
    outputs = tf.keras.layers.Add()([outputs, inputs[:, 1:]])

    #outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]])
    # outputs = tf.keras.layers.Add()([outputs, inputs[:, :out_dim]]) NN4UQ
    #    out = keras.layers.Dense(units=3, activation= "linear")(added)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    optimizer = optimizers.Adagrad(lr=learning_rate)# amsgrad=True)
    model.compile(optimizer=optimizer,
                  loss='mean_absolute_error')
    return model

In [ ]:
get_custom_objects().update({'swish': Activation(swish)})


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(Xdnn, Ydnn, shuffle=True, train_size=0.8, random_state=42)
in_dim = X_train.shape[1]
out_dim = Y_train.shape[1]

In [ ]:
out_dim

3

In [ ]:
np.max(X_test[:,0])

14.958860320136234

In [ ]:
#print(X_train.shape)
learning_rate=0.05#gp_result.x[0]
num_layers = 6#gp_result.x[1]
num_nodes = 4 #gp_result.x[2]
batch_size=10#gp_result.x[3]
epochs=300
activation='swish'
DropModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                             activation=activation, learning_rate=learning_rate,
                             training=True)  # True-activates dropout during prediction
DropModel.summary()
DropModel.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data = (X_test, Y_test))

DetModel = get_model_layers(nodes=num_nodes, layers=num_layers,
                            activation=activation, learning_rate=learning_rate)
#name = '{}_{}_{}_{}'.format(num_nodes, num_layers, activation, learning_rate)
#DropModel.save(root + 'results/nc/' + name)
DetModel.set_weights(DropModel.get_weights())
score = DetModel.evaluate(X_test, Y_test, verbose=0)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 4)            20          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 4)            20          dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 4)            20          dense_1[0][0]                    
______________________________________________________________________________________________

In [ ]:
'''
#Simulating both systems using some initial condition
#The predictions will be quite off since the difference in the models propagates within a trajectory
F=[]
tarr=[]
Tsamp = 0.005
Nsamp = 50
F = np.random.uniform(low=5, high=15, size=(Nsamp,))
tarr = np.zeros((Nsamp,1))

#Ca_plant,Cb_plant,Tr_plant,Tk_plant = PlantSimulator(Tsamp,Nsamp,F)
#Get states for a sequence of F, given the models
Cx_model,Ca_model,Cp_model = ModelSimulator(Tsamp,Nsamp,F1)
Cx_plant,Ca_plant,Cp_plant = PlantSimulator(Tsamp,Nsamp,F1)
Cx_resnet,Ca_resnet,Cp_resnet = ResnetModelSimulator(Tsamp,Nsamp,F1,DropModel)





for j in range(Nsamp):
    tarr[j] = j*Tsamp

tarr = np.append(tarr, np.repeat(np.nan, 1))
F = np.append(F, np.repeat(np.nan, 1))

plt.figure(dpi=50)
plt.plot(Cx_plant,'r--',label='Plant',linewidth=3.0,alpha=0.8)
plt.plot(Cx_model,'g--',label='HF Model',linewidth=6.0)
plt.plot(Cx_resnet,'b-',label='ResNET',linewidth=3.0)
plt.xlabel("Sampling step")
plt.ylabel("Biomass")
plt.legend()

plt.figure(dpi=50)
plt.plot(Ca_plant,'r--',label='Plant',linewidth=3.0,alpha=0.8)
plt.plot(Ca_model,'g--',label='HF Model',linewidth=6.0)
plt.plot(Ca_resnet,'b-',label='ResNET',linewidth=3.0)
plt.xlabel("Sampling step")
plt.ylabel("Acetate")
plt.legend()


plt.figure(dpi=50)
plt.plot(Cp_plant,'r--',label='Plant',linewidth=3.0,alpha=0.8)
plt.plot(Cp_model,'g--',label='HF Model',linewidth=6.0)
plt.plot(Cp_resnet,'b-',label='ResNET',linewidth=3.0)
plt.xlabel("Sampling step")
plt.ylabel("Hydrogen")
plt.legend()

plt.figure(dpi=50)
plt.plot(F1,linewidth=3.0)
plt.legend()
plt.xlabel("Sampling step")
plt.ylabel("Flow Rate")
plt.legend()
'''

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


ValueError: ignored

Some preliminary work to extract weights; Ignore

The above are open-loop predictions

In [ ]:
DropModel.summary()

In [ ]:
weights = DropModel.get_weights() # returs a numpy list of weights


In [ ]:
weights

In [ ]:
#DropModel.set_weights(weights)

Point-wise predictions for MPC

In [1]:
def eulerPlantNext(z):
    #next states for plant
    #Define constants
    F = z[0]
    Cx = z[1]
    Ca = z[2]
    Cp = z[3]
    print(F,Ca,Cx,Cp)
    Tsamp=0.005;
    mumax =40
    Ks =2e-1
    Yx =0.04
    Yp =0.8
    pstar=1.8
    Ca0=10.0

    kd= 1e-2;
    aa = 5.5;
    bb = 8e-1;
    
        
    if Cp>pstar:
       mu = 0
    else:
       mu = mumax*(Ca/(Ca+Ks))*(1-Cp/pstar)**3
  
        

    #Rate Definitions
    fcx=(mu-kd)*Cx - F*Cx
    fcp=aa*fcx +bb*Cx- F*Cp
    fca=-(1/Yx)*fcx - (1/Yp)*fcp + F*(Ca0-Ca)
   
    Cx_kplusone = Cx + Tsamp*fcx
    Ca_kplusone = Ca + Tsamp*fca
    Cp_kplusone = Cp + Tsamp*fcp
    
    
    return Cx_kplusone, Ca_kplusone,Cp_kplusone

In [ ]:
uk=np.array([10.0]) #Control Input
xk=np.array([0.2,15,0]) #Current States
zk=np.concatenate((uk,xk)) #

In [ ]:
xkplusone_neural = DropModel.predict(zk.reshape(1,4))
print('Xkplusone from residual net:',xkplusone_neural)
xkplusone_plant  = eulerPlantNext(zk)
print('Xkplusone from plant:',xkplusone_plant)


In [ ]:
# def dropModelCustom():